<a href="https://colab.research.google.com/github/GauravGupta7/project_S/blob/master/Linear_regression_using_pytorch_builtins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This model is built using pytorch built-ins. It implements the concept of Linear Regression. 

Refer : (https://jovian.ai/aakashns/02-linear-regression)

We are importing the torch.nn package which has built-ins for working with neural networks. 

In [26]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [27]:
#Setting up the input and target Matrices
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [28]:
# Define dataset
train_ds = TensorDataset(inputs, targets)

In [29]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
print(train_dl)

In [30]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 134.,  58.],
        [ 73.,  66.,  44.],
        [ 91.,  87.,  65.],
        [ 87., 135.,  57.],
        [101.,  44.,  37.]])
tensor([[119., 133.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 134.],
        [ 21.,  38.]])


In [31]:
#Defining the model using "nn.linear" class
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.2308,  0.0957,  0.1389],
        [ 0.2800, -0.1378, -0.2318]], requires_grad=True)
Parameter containing:
tensor([-0.2449,  0.5380], requires_grad=True)


Pytorch has a built in function '.parameters()' which returns all the weights and bias martices present in the model

In [32]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2308,  0.0957,  0.1389],
         [ 0.2800, -0.1378, -0.2318]], requires_grad=True),
 Parameter containing:
 tensor([-0.2449,  0.5380], requires_grad=True)]

In [33]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 28.9852,   1.7822],
        [ 38.0658,  -0.9380],
        [ 40.7097,  -7.0063],
        [ 32.5477,  14.6008],
        [ 34.5880,  -9.5916],
        [ 29.1203,   2.2000],
        [ 38.1091,  -1.0319],
        [ 41.0794,  -6.9581],
        [ 32.4126,  14.1830],
        [ 34.4962, -10.1034],
        [ 29.0285,   1.6882],
        [ 38.2009,  -0.5201],
        [ 40.6664,  -6.9124],
        [ 32.6396,  15.1126],
        [ 34.4529, -10.0095]], grad_fn=<AddmmBackward>)

**LOSS FUNCTION:**
Instead of defining a loss function manually, we use built ins to do the part. The function used is mse_loss 

In [34]:
# Define loss function
loss_fn = F.mse_loss

Computing the loss for current prediction

In [35]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(6509.1504, grad_fn=<MseLossBackward>)


**Optimizer:** Instead of manually working upon the bias we use optim.SGD optimizer of pytorch. SGD stands for stochastic gradient descent. The term stochastic means that the samples are selected in random batches instead of as a single group. The model.parameters() is passed as an argument to optim.SGD so that the optimizer knows which matrices should be modified during the update step. We can specify a learning rate that controls the amount by which the parameters are modified.

In [36]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

**Training The Model-**

Here we define a utility function 'fit' that trains the model for a given number of epochs.

Some things to note above:

1. We use the data loader defined earlier to get batches of data for every iteration.

2. Instead of updating parameters (weights and biases) manually, we use opt.step to perform the update and opt.zero_grad to reset the gradients to zero.

3. We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. loss.item returns the actual value stored in the loss tensor.

In [37]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [38]:
#Train the model for 100 epochs
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 596.2767
Epoch [20/100], Loss: 312.3689
Epoch [30/100], Loss: 263.3254
Epoch [40/100], Loss: 88.4079
Epoch [50/100], Loss: 175.3555
Epoch [60/100], Loss: 9.8580
Epoch [70/100], Loss: 119.2938
Epoch [80/100], Loss: 27.4209
Epoch [90/100], Loss: 28.7920
Epoch [100/100], Loss: 33.1440


In [39]:
#After training again generate predictions:
preds = model(inputs)
print(loss_fn(preds, targets))

tensor(37.2483, grad_fn=<MseLossBackward>)


In [40]:
#To predict for a single data:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.8890, 68.9377]], grad_fn=<AddmmBackward>)